In [223]:
import sqlalchemy as db
import pandas as pd

from sqlalchemy import Integer, ForeignKey, String, Column
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship

In [224]:
engine = db.create_engine('sqlite:///scraped_jobs.sqlite')

In [225]:
connection = engine.connect()


In [226]:
# Base = declarative_base()

# class 

In [227]:
metadata = db.MetaData()

salaries = db.Table('salaries', metadata,
                   db.Column('Job_Id', db.Integer(), primary_key=True),
                   db.Column('Job_Name', db.String(255), nullable = False),
                   db.Column('Org_Name', db.String(255), nullable = True),
                   db.Column('Salary', db.Integer()))

salary_period = db.Table('salary_period', metadata,
                   db.Column('Job_Id', db.Integer(), foreign_key=True),
                   db.Column('Salary_Time_Period', db.String(255), nullable = True),
                   db.Column('Salary_Converted_Hourly', db.Integer()))


job_search = db.Table('job_search', metadata,
                   db.Column('Job_Id', db.Integer(), foreign_key=True),
                   db.Column('Searched_data_scientist', db.Boolean()),
                   db.Column('Searched_machine_learning_engineer', db.Boolean()),
                   db.Column('Searched_data_analyst', db.Boolean(), ))

metadata.create_all(engine)

/Users/ipreoteasa/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/sqlalchemy/sql/base.py:302: SAWarning: Can't validate argument 'foreign_key'; can't locate any SQLAlchemy dialect named 'foreign'
  % (k, dialect_name)


In [228]:
t = pd.read_sql_query('''SELECT s.Job_Id, Job_Name, Salary, Salary_Time_Period 
                      FROM salaries s JOIN salary_period p ON s.Job_Id = p.Job_Id
                      WHERE Salary_Time_Period == 'Y' ''', connection)
t

,Job_Id,Job_Name,Salary,Salary_Time_Period


In [229]:
df = pd.read_csv('../clean_data/jobs.csv', index_col=0)
# df.rename({'Unnamed: 0': 'Job Id'},)
df.head(2)

,company,job_title,job_descr,job_url,job_location,searched_data_analyst,searched__data_scientist,searched__machine_learning_engineer,salary_from_page_source_as_stated,salary_from_page_source_conv_hourly,salary_from_page_source_time_period,Num_reviews,id
0,Medicines and Healthcare products Regulatory A...,CPRD Data Scientist,would like reassure all applicant that recruit...,https://www.indeed.co.uk/pagead/clk?mo=r&ad=-6...,NaN,0,1,0,39853.0,21.0,Y,18.0,0
1,PwC,Deals - Investigative Analytics - Data Scienti...,career within Forensics Technology service wil...,https://www.indeed.co.uk/rc/clk?jk=7548e60a487...,NaN,0,1,0,NaN,NaN,NaN,7678.0,1


In [230]:
salaries_df = df[['id', 'job_title', 'company', 'salary_from_page_source_as_stated']]
salaries_df.rename({'id': 'Job_Id', 'job_title': 'Job_Name', 'company':'Org_Name', 'salary_from_page_source_as_stated':'Salary'}, axis=1, inplace=True)
salaries_df.head()

/Users/ipreoteasa/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Job_Id,Job_Name,Org_Name,Salary
0,0,CPRD Data Scientist,Medicines and Healthcare products Regulatory A...,39853.0
1,1,Deals - Investigative Analytics - Data Scienti...,PwC,NaN
2,2,Data Scientist - Cyber,Capital One - UK,NaN
3,3,Lead Data Scientist,Globant,NaN
4,4,G4 Lead Data Scientist - Cyber,UK Government - National Crime Agency,35846.0


In [231]:
salaries_df.to_sql('salaries', 
                  connection,
                  if_exists='append',index=False)

In [232]:
period_df = df[['id', 'salary_from_page_source_conv_hourly', 'salary_from_page_source_time_period']]
period_df.rename({'id': 'Job_Id', 'salary_from_page_source_conv_hourly': 'Salary_Converted_Hourly', 'salary_from_page_source_time_period':'Salary_Time_Period'}, axis=1, inplace=True)
period_df.head(2)

/Users/ipreoteasa/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Job_Id,Salary_Converted_Hourly,Salary_Time_Period
0,0,21.0,Y
1,1,NaN,NaN


In [233]:
period_df.dropna(inplace=True)
period_df.head()

/Users/ipreoteasa/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Job_Id,Salary_Converted_Hourly,Salary_Time_Period
0,0,21.00,Y
4,4,18.89,Y
7,7,42.15,Y
8,8,13.17,Y
11,11,80.00,H


In [234]:
period_df.to_sql('salary_period', 
                  connection,
                  if_exists='append',index=False)

In [235]:
search_df = df[['id', 'searched__data_scientist', 'searched_data_analyst', 'searched__machine_learning_engineer']]
search_df.rename({'id': 'Job_Id', 'searched__data_scientist': 'Searched_data_scientist', 
                  'searched_data_analyst': 'Searched_data_analyst', 
                  'searched__machine_learning_engineer':'Searched_machine_learning_engineer'}, axis=1, inplace=True)
search_df.head(2)

/Users/ipreoteasa/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Job_Id,Searched_data_scientist,Searched_data_analyst,Searched_machine_learning_engineer
0,0,1,0,0
1,1,1,0,0


In [236]:
search_df.to_sql('job_search', connection, if_exists='append', index=False)

In [237]:
t.loc[t.Job_Name.str.contains('(?<![\w\d])Lead(?![\w\d])')]

,Job_Id,Job_Name,Salary,Salary_Time_Period


In [238]:
pd.read_sql_query("""SELECT * FROM job_search""", connection)

,Job_Id,Searched_data_scientist,Searched_machine_learning_engineer,Searched_data_analyst
0,0,1,0,0
1,1,1,0,0
2,2,1,0,0
3,3,1,0,0
4,4,1,0,0
...,...,...,...,...
2206,2206,0,0,1
2207,2207,0,0,1
2208,2208,0,0,1
2209,2209,0,0,1


In [264]:
pd.read_sql_query('''SELECT s.Job_Id, Job_Name, Org_Name, Salary                                 
                            FROM salaries s JOIN job_search j ON s.Job_Id == j.Job_Id 
                            JOIN salary_period p on s.Job_Id == p.Job_Id
                            WHERE Salary IS NOT NULL AND 
                                   (j.Searched_data_scientist == 1 AND
                                   j.Searched_data_analyst == 1 AND
                                   j.Searched_machine_learning_engineer == 0) 
                                   
                                   AND
                                   p.Salary_Time_Period == 'Y' 
                            LIMIT 1000
                             ''', 
                  connection)

,Job_Id,Job_Name,Org_Name,Salary,Searched_data_scientist
